In [5]:
import pandas as pd
import os
from tqdm import tqdm
import numpy as np
import re
import geopandas as gpd

import warnings
warnings.filterwarnings('ignore')

# 1. Препроцессинг паркетов

In [6]:
list_of_files = list()
for (dirpath, dirnames, filenames) in os.walk("../../data/01_raw/second_departure_parquet/"):
    list_of_files += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [3]:
def preprocess_accelerometer_data(one_row_acc_data):
    possible_list = eval(one_row_acc_data) if one_row_acc_data is not None else [np.nan, np.nan, np.nan]
    if len(possible_list) == 0:
        possible_list = [np.nan, np.nan, np.nan]
    return possible_list

In [4]:
for filepath in tqdm(list_of_files):
    data = pd.read_parquet(filepath)
    data["timestamp_rownum"] = data.groupby(["Время отправки гео"]).cumcount() + 1
    data['Акселерометр'] = data['Акселерометр'].str.replace("\n", "").apply(lambda x: preprocess_accelerometer_data(x))
    data[["acc_x", "acc_y", "acc_z"]] = data['Акселерометр'].to_list()
    data.to_parquet(filepath.replace('second_departure_parquet', 'second_departure_parquet_preprocessed').replace(" ", "_"))

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [04:35<00:00,  2.68s/it]


In [5]:
list_of_files_with_anomalies = list()
for (dirpath, dirnames, filenames) in os.walk("../../data/01_raw/second_departure_parquet_preprocessed/Данные_за_06.09-12.09/"):
    list_of_files_with_anomalies += [os.path.join(dirpath, file) for file in filenames if ".parquet" in file]

In [6]:
def get_date_from_filename(filename):
    filename_list = filename.replace('.parquet', '').split("_")
    for each in filename_list:
        found_date = re.search("[0-9][0-9](.)[0-9][0-9](.)20[0-9][0-9]$", each)
        if found_date is not None:
            return found_date.group()

In [7]:
dict_dates = dict()
for filename in tqdm(list_of_files_with_anomalies):
    date = get_date_from_filename(filename)
    if date not in dict_dates:
        dict_dates[date] = []
    df = pd.read_parquet(filename)
    dict_dates[date].append(df)

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [00:07<00:00,  9.97it/s]


In [8]:
def make_subsample_for_df(df):
    subsample = df.sample(n=50)
    dev_id = subsample["Наименование (device ID)"].values[0]
    gdf_subsample = gpd.GeoDataFrame(subsample, 
                                     geometry=gpd.points_from_xy(subsample['Долгота'], subsample['Широта']), 
                                     crs=4326).to_crs(3857)
    data_for_gdf = d = {"dev_id": [dev_id], "geometry":[gdf_subsample.buffer(1).unary_union]}
    output_gdf = gpd.GeoDataFrame(data_for_gdf, crs=3857)
    return output_gdf

In [9]:
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    dt_column = "Время отправки гео"
    device_id_column = "Наименование (device ID)"
    if not df[dt_column].is_monotonic_increasing:
        df = df.sort_values(by=[dt_column], ignore_index=True)
    df = df.groupby([device_id_column, dt_column]).head(1)
    df = df.sort_values(by=[dt_column], ignore_index=True)
    return df

In [10]:
def convert_dataframe_to_geodataframe(df: pd.DataFrame) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Широта'], df['Долгота']), 
                                     crs=4326).to_crs(32633)
    gdf["x_metres"] = gdf["geometry"].x
    gdf["y_metres"] = gdf["geometry"].y
    return gdf

In [11]:
def make_subsample_for_df(df):
    subsample = df.sample(n=50)
    dev_id = subsample["Наименование (device ID)"].values[0]
    gdf_subsample = gpd.GeoDataFrame(subsample, 
                                     geometry=gpd.points_from_xy(subsample['Долгота'], subsample['Широта']), 
                                     crs=4326).to_crs(3857)
    data_for_gdf = d = {"dev_id": [dev_id], "geometry":[gdf_subsample.buffer(1).unary_union]}
    output_gdf = gpd.GeoDataFrame(data_for_gdf, crs=32633)
    return output_gdf

In [12]:
def find_possible_intersections(df: gpd.GeoDataFrame):
    df_copy = df.copy(deep=True)
    df_copy["geometry"] = df_copy.buffer(50)
    data_for_gdf = {"dev_id": [], 
                    "geometry": []}
    for unique_id in df_copy["Наименование (device ID)"].unique():
        data_for_gdf["dev_id"].append(unique_id)
        data_for_gdf["geometry"].append(df_copy[df_copy["Наименование (device ID)"] == unique_id].unary_union)
    batch_polygons = gpd.GeoDataFrame(data_for_gdf, crs=32633)
    possible_intersects = batch_polygons.sjoin(batch_polygons, how='inner', predicate="intersects")
    possible_intersects = possible_intersects[possible_intersects["dev_id_left"] !=
                                              possible_intersects["dev_id_right"]]
    return possible_intersects

In [13]:
def drop_permutations_copys(df: pd.DataFrame) -> pd.DataFrame:
    cols = df.columns.values
    df.loc[:, cols] = np.sort(df.loc[:, cols].values, axis=1)
    df = df.drop_duplicates()
    return df

In [14]:
def count_speed(row):
    x_diff_1 = row["x_metres_x_left"] - row["x_metres_x_right"]
    y_diff_1 = row["y_metres_x_left"] - row["y_metres_x_right"]
    x_diff_2 = row["x_metres_y_left"] - row["x_metres_y_right"]
    y_diff_2 = row["y_metres_y_left"] - row["y_metres_y_right"]
    time_diff = (row["Время отправки гео_left"] - row["Время отправки гео_right"]).total_seconds()
    
    distance_1 = np.sqrt(x_diff_1 ** 2 + y_diff_1 ** 2)
    distance_2 = np.sqrt(x_diff_2 ** 2 + y_diff_2 ** 2)
    row["speed_left"] = distance_1 / time_diff
    row["speed_right"] = distance_2 / time_diff
    return row


def count_correlation_features(merged_df: gpd.GeoDataFrame, 
                               data_df: pd.DataFrame):
    accelerometer_axis = ["x", "y", "z"]
    for axis in accelerometer_axis:
        data_df[f"accelerometer_{axis}_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].corr().values[1, 0])
        data_df[f"accelerometer_{axis}_diff_corr"] = abs(merged_df[[f"acc_{axis}_x", f"acc_{axis}_y"]].diff(1).corr().values[1, 0])
        
    metres_columns_correlation = ["x", "y"]
    for meter_column in metres_columns_correlation:
        data_df[f"{meter_column}_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].corr().values[1, 0])
        data_df[f"{meter_column}_diff_metres_corr"] = abs(merged_df[[f"{meter_column}_metres_x", f"{meter_column}_metres_y"]].diff(1).corr().values[1, 0])
        
    data_df["height_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].corr().values[1, 0])
    data_df["bearing_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].corr().values[1, 0])
    data_df["speed_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].corr().values[1, 0])
    
    data_df["height_diff_corr"] = abs(merged_df[["Высота_x", "Высота_y"]].diff(1).corr().values[1, 0])
    data_df["bearing_diff_corr"] = abs(merged_df[["bearing_x", "bearing_y"]].diff(1).corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_df[["Скорость_x", "Скорость_y"]].diff(1).corr().values[1, 0])
    
    merged_itself = merged_df.join(merged_df.shift(), lsuffix="_left", rsuffix="_right")
    merged_itself.reset_index(drop=True, inplace=True)
    merged_itself = merged_itself.loc[1:].apply(lambda x: count_speed(x), axis=1)
    
    data_df["speed_corr"] = abs(merged_itself[["speed_left", "speed_right"]].corr().values[1, 0])
    data_df["speed_diff_corr"] = abs(merged_itself[["speed_left", "speed_right"]].diff(1).corr().values[1, 0])
    
    data_df["speed_mean_abs_diff"] = abs(merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_mean_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).mean()
    data_df["speed_std_diff"] = (merged_itself["speed_left"] - merged_itself["speed_right"]).std()
    return data_df


def get_features_from_dataframes(gdf1: gpd.GeoDataFrame, 
                                 gdf2: gpd.GeoDataFrame):
    data = pd.DataFrame()
    merged_df = gdf1.merge(gdf2, on=["Время отправки гео"], how="inner")
    data["date"] = [gdf1["Время отправки гео"].dt.to_period('D').values[0].to_timestamp()]
    data["datetime"] = gdf1.reset_index()["time_category"][0].left
    data["dev_id_1"] = gdf1["Наименование (device ID)"].values[0]
    data["id_1_last_geometry"] = gdf1.tail(1).reset_index()["geometry"][0]
    data["id_1_last_lon"] = gdf1.tail(1).reset_index()["Долгота"][0]
    data["id_1_last_lat"] = gdf1.tail(1).reset_index()["Широта"][0]
    data["dev_id_2"] = gdf2["Наименование (device ID)"].values[0]
    data["id_2_last_geometry"] = gdf2.tail(1).reset_index()["geometry"][0]
    data["id_2_last_lon"] = gdf2.tail(1).reset_index()["Долгота"][0]
    data["id_2_last_lat"] = gdf2.tail(1).reset_index()["Широта"][0]
    if merged_df.shape[0] < 5:
        return data
    
    merged_df['distance'] = merged_df.apply(lambda x: x["geometry_x"].distance(x["geometry_y"]), axis=1)
    
    data = count_correlation_features(merged_df, data)
    data["distance_mean"] = merged_df['distance'].mean()
    data["distance_std"] = merged_df['distance'].std()
    return data

# final features collection

In [15]:
# Добавить фичи по корреляции координат, высоты, наклона, скорости 
# фичи по приращению координат, акселерометра

In [16]:
dates = dict_dates.keys()
feats = []
for date in tqdm(dates):
    data_list = dict_dates[date]
    all_data = pd.concat(data_list, ignore_index=True)
    all_data_preprocessd = preprocess_dataframe(all_data)
    all_data_gdf = convert_dataframe_to_geodataframe(all_data_preprocessd)
    cuts = pd.date_range(all_data_gdf["Время отправки гео"].min() - pd.Timedelta("5min"),
                     all_data_gdf["Время отправки гео"].max() + pd.Timedelta("5min"),
                     freq="5min")
    all_data_gdf["time_category"] = pd.cut(all_data_gdf["Время отправки гео"], cuts)
    useful_samples = []
    for each in all_data_gdf["time_category"].unique():
        tmp_df = all_data_gdf[all_data_gdf["time_category"] == each]
        if tmp_df["Наименование (device ID)"].nunique() > 2:
            useful_samples.append(tmp_df)
    for sample in useful_samples:
        possible_intersecions = find_possible_intersections(sample)
        relevant_pairs = possible_intersecions[["dev_id_left", "dev_id_right"]]
        relevant_pairs = drop_permutations_copys(relevant_pairs)
        for i, (first_id, second_id) in relevant_pairs.iterrows():
            df1 = sample[sample["Наименование (device ID)"] == first_id]
            df2 = sample[sample["Наименование (device ID)"] == second_id]
            feats.append(get_features_from_dataframes(df1, df2))
        break
    break

  0%|                                                                                            | 0/5 [01:11<?, ?it/s]


In [24]:
import numpy as np

In [57]:
concat_gdf = gpd.GeoDataFrame(pd.concat(feats, ignore_index=True))

In [61]:
concat_gdf_copy = concat_gdf.drop(["id_1_last_geometry", "id_2_last_geometry"], axis=1)

In [63]:
concat_gdf_copy.to_parquet("../../data/02_interim/sample_06_12_all_features.parquet")

# Ниже вам не надо

In [1]:
import pandas as pd

In [3]:
aa = pd.read_parquet("../../data/02_interim/sample_06_12_all_features.parquet")

In [4]:
aa

,date,datetime,dev_id_1,id_1_last_lon,id_1_last_lat,dev_id_2,id_2_last_lon,id_2_last_lat,accelerometer_x_corr,accelerometer_x_diff_corr,...,bearing_corr,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std
0,2022-09-06,2022-09-06 07:38:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602788,55.893562,3744EED3-A474-458B-AA4F-5357B8EE7FBF,37.606185,55.894089,0.000661,0.035777,...,0.205053,0.082422,1.421230e-02,0.098073,0.029655,3.504287,-1.845214,9.843380,165.717646,141.099882
1,2022-09-06,2022-09-06 07:38:07,3744EED3-A474-458B-AA4F-5357B8EE7FBF,37.606185,55.894089,393B6105-4D72-4E11-BEA1-394CD0AFC499,37.604149,55.895599,0.002194,0.077567,...,0.306874,0.012220,1.329316e-03,0.043845,0.007309,4.445489,0.413304,11.253297,88.975281,98.468099
2,2022-09-06,2022-09-06 07:38:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602788,55.893562,393B6105-4D72-4E11-BEA1-394CD0AFC499,37.604149,55.895599,0.029039,0.067258,...,0.144063,0.052211,1.114944e-02,0.152548,0.059629,2.732987,-1.196340,6.964679,133.042537,72.842306
3,2022-09-06,2022-09-06 07:48:07,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,37.602780,55.893606,7F921169-FB12-471F-B244-62840E543DD7,37.603513,55.892299,0.024953,0.069213,...,0.046329,0.009875,1.156702e-02,0.269572,0.014251,3.261829,-0.938763,8.793781,118.765372,16.460189
4,2022-09-06,2022-09-06 08:13:07,40808752-2CEE-454B-B6E6-95EC9615999D,37.603324,55.892073,AE6C3F14-5263-4BF1-AD06-3A8CE0A27AAB,37.603368,55.892132,0.157693,0.012551,...,0.630734,0.023942,5.139841e-03,0.754217,0.042451,4.843979,1.013350,12.424083,17.029361,15.709145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6547,2022-09-12,2022-09-12 17:21:39,2D7135B3-1208-45BA-9B5A-0FC27C13B9B0,37.611783,55.892302,329C29F6-A464-4174-867E-50D7D163206C,37.610464,55.891030,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6548,2022-09-12,2022-09-12 17:21:39,0699B415-A73E-4CFA-B910-079AA0408B1B,37.611192,55.887478,40808752-2CEE-454B-B6E6-95EC9615999D,37.610962,55.887849,0.793525,0.601976,...,NaN,0.665669,NaN,NaN,0.619808,3.918556,2.154666,7.304032,98.934811,30.686367
6549,2022-09-12,2022-09-12 17:26:39,329C29F6-A464-4174-867E-50D7D163206C,37.610725,55.890972,55464DF0-C385-4349-801B-C8E1CB861CB5,37.610891,55.890427,0.025361,0.161036,...,0.246894,0.070157,1.767547e-18,0.000059,0.050241,3.430670,-1.592060,9.831637,65.952005,7.381049
6550,2022-09-12,2022-09-12 17:26:39,0699B415-A73E-4CFA-B910-079AA0408B1B,37.611192,55.887478,40808752-2CEE-454B-B6E6-95EC9615999D,37.611027,55.887835,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
pd.concat(feats)

,date,dev_id_1,dev_id_2,accelerometer_x_corr,accelerometer_x_diff_corr,accelerometer_y_corr,accelerometer_y_diff_corr,accelerometer_z_corr,accelerometer_z_diff_corr,x_metres_corr,...,bearing_corr,speed_corr,height_diff_corr,bearing_diff_corr,speed_diff_corr,speed_mean_abs_diff,speed_mean_diff,speed_std_diff,distance_mean,distance_std
0,2022-09-06,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,3744EED3-A474-458B-AA4F-5357B8EE7FBF,0.000661,0.067258,0.018514,0.005505,0.009090,0.066659,0.370997,...,0.205053,0.082422,1.421230e-02,0.098073,0.029655,3.504287,-1.845214,9.843380,165.717646,141.099882
0,2022-09-06,3744EED3-A474-458B-AA4F-5357B8EE7FBF,393B6105-4D72-4E11-BEA1-394CD0AFC499,0.002194,0.067258,0.079801,0.005505,0.311329,0.066659,0.572543,...,0.306874,0.012220,1.329316e-03,0.043845,0.007309,4.445489,0.413304,11.253297,88.975281,98.468099
0,2022-09-06,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,393B6105-4D72-4E11-BEA1-394CD0AFC499,0.029039,0.067258,0.079644,0.005505,0.056285,0.066659,0.899834,...,0.144063,0.052211,1.114944e-02,0.152548,0.059629,2.732987,-1.196340,6.964679,133.042537,72.842306
0,2022-09-06,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,7F921169-FB12-471F-B244-62840E543DD7,0.024953,0.067258,0.046449,0.005505,0.050282,0.066659,0.324171,...,0.046329,0.009875,1.156702e-02,0.269572,0.014251,3.261829,-0.938763,8.793781,118.765372,16.460189
0,2022-09-06,40808752-2CEE-454B-B6E6-95EC9615999D,AE6C3F14-5263-4BF1-AD06-3A8CE0A27AAB,0.157693,0.067258,0.614122,0.005505,0.305235,0.066659,0.923407,...,0.630734,0.023942,5.139841e-03,0.754217,0.042451,4.843979,1.013350,12.424083,17.029361,15.709145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-09-12,2D7135B3-1208-45BA-9B5A-0FC27C13B9B0,329C29F6-A464-4174-867E-50D7D163206C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,2022-09-12,0699B415-A73E-4CFA-B910-079AA0408B1B,40808752-2CEE-454B-B6E6-95EC9615999D,0.793525,0.067258,0.402093,0.005505,NaN,0.066659,0.691097,...,NaN,0.665669,NaN,NaN,0.619808,3.918556,2.154666,7.304032,98.934811,30.686367
0,2022-09-12,329C29F6-A464-4174-867E-50D7D163206C,55464DF0-C385-4349-801B-C8E1CB861CB5,0.025361,0.067258,0.057308,0.005505,0.030528,0.066659,0.286634,...,0.246894,0.070157,1.767547e-18,0.000059,0.050241,3.430670,-1.592060,9.831637,65.952005,7.381049
0,2022-09-12,0699B415-A73E-4CFA-B910-079AA0408B1B,40808752-2CEE-454B-B6E6-95EC9615999D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
dd = pd.concat(feats)

In [154]:
sample

,Наименование (device ID),Время отправки гео,bearing,Акселерометр,Гироскоп,Счетчик шагов,Скорость,"Точность, м",Долгота,Широта,Высота,Уровень заряда,timestamp_rownum,acc_x,acc_y,acc_z,geometry,x_metres,y_metres,time_category
361724,2D7135B3-1208-45BA-9B5A-0FC27C13B9B0,2022-09-06 15:58:08,45,"[0.6489105, -9.584518, -1.2789001]",[],3728.0,3.671730,15,37.612080,55.892171,184.70000,53,5,0.648910,-9.584518,-1.278900,POINT (4167106.738 5047346.294),4.167107e+06,5.047346e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
361725,B94A4D90-F127-487B-9373-5FEE1A34A04F,2022-09-06 15:58:08,66,"[3.61615, -9.084503, -0.2666626]",[],6106.0,0.278239,23,37.612703,55.888690,179.40000,63,1,3.616150,-9.084503,-0.266663,POINT (4166751.211 5047249.609),4.166751e+06,5.047250e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
361726,95D12D18-BCB5-4016-B4CC-EAF7195FDC8A,2022-09-06 15:58:08,0,"[-3.2908478, 8.297043, 4.2724457]",[],13070.0,0.000000,16,37.599950,55.891763,188.40000,54,5,-3.290848,8.297043,4.272446,POINT (4167806.092 5045934.929),4.167806e+06,5.045935e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
361727,87104440-8B6E-4874-992B-5CACCAB5E31D,2022-09-06 15:58:08,252,"[-7.756607, 3.2579193, -4.870102]",[],856.0,0.000113,500,37.603410,55.893414,173.33570,53,1,-7.756607,3.257919,-4.870102,POINT (4167746.658 5046411.658),4.167747e+06,5.046412e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
361728,1CB8D3E9-04BE-465D-B334-0E1F7F026A98,2022-09-06 15:58:08,281,"[2.3855286, 8.948334, -2.1625671]",[],9643.0,0.268401,23,37.608332,55.893280,185.09999,37,1,2.385529,8.948334,-2.162567,POINT (4167435.517 5046969.871),4.167436e+06,5.046970e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
366250,2D7135B3-1208-45BA-9B5A-0FC27C13B9B0,2022-09-06 16:03:07,58,"[-1.1732025, -9.871246, 0.17739868]",[],3731.0,0.503609,17,37.612011,55.892266,184.70000,52,1,-1.173202,-9.871246,0.177399,POINT (4167119.602 5047342.984),4.167120e+06,5.047343e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
366251,95D12D18-BCB5-4016-B4CC-EAF7195FDC8A,2022-09-06 16:03:07,0,"[-2.3944702, 9.287018, 2.4969025]",[],13070.0,0.000000,19,37.599931,55.891760,188.40000,53,1,-2.394470,9.287018,2.496903,POINT (4167806.933 5045932.568),4.167807e+06,5.045933e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
366252,38702660-F4AA-472F-8FAB-B3B47B5889AC,2022-09-06 16:03:07,0,"[6.0169373, -8.075333, 1.284195]",[],2808.0,0.000000,273,37.612633,55.888562,177.90000,49,1,6.016937,-8.075333,1.284195,POINT (4166743.718 5047235.465),4.166744e+06,5.047235e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"
366253,3744EED3-A474-458B-AA4F-5357B8EE7FBF,2022-09-06 16:03:07,83,"[-5.212158, 7.794342, 2.4388885]",[],16675.0,0.066394,21,37.616812,55.891131,177.70000,60,2,-5.212158,7.794342,2.438889,POINT (4166724.418 5047838.999),4.166724e+06,5.047839e+06,"(2022-09-06 15:58:07, 2022-09-06 16:03:07]"


In [26]:
bb = gpd.GeoDataFrame(pd.concat(aa, ignore_index=True), crs=3857)

In [27]:
bb["geometry"][0].area

100.46614168204151

In [29]:
joined = bb.sjoin(bb, how='inner', predicate="intersects")

In [31]:
joined[joined["dev_id_left"] != joined["dev_id_right"]].shape

(58, 4)

In [242]:
cc = bb.sjoin(droped_df, how='inner', predicate='intersects')

In [263]:
import gc 

del ff
gc.collect()

985

In [248]:
ee = dd.merge(dd, on=["Время отправки гео", "timestamp_rownum"])

In [250]:
ee = ee[ee["Наименование (device ID)_x"] != ee["Наименование (device ID)_y"]]

In [254]:
ee = ee.sort_values(by=["Наименование (device ID)_x", "Наименование (device ID)_y", 
                   "Время отправки гео", "timestamp_rownum"])

In [276]:
ee1 = ee[(ee["Наименование (device ID)_x"] == "0699B415-A73E-4CFA-B910-079AA0408B1B") & 
   (ee["Наименование (device ID)_y"] == "294A06F3-F7FE-4AAC-8099-049733BA1315")][["acc_x_x", "acc_x_y"]]

In [48]:
data_1 = pd.read_parquet("../../data/01_raw/second_departure_parquet/Данные за 06.09-12.09/Тест 5hz - 6/Тест 5 Hz -6 06.09.2022.parquet")
data_2 = pd.read_parquet("../../data/01_raw/second_departure_parquet/Данные за 06.09-12.09/Тест 5hz - 6/Тест 5hz - 6 06.09.2022 2-е устройство.parquet")
data_3 = pd.read_parquet("../../data/01_raw/second_departure_parquet/Данные за 06.09-12.09/Тест 5hz - 5/Тест 5 Hz -5 06.09.2022.parquet")

In [49]:
data_1["timestamp_rownum"] = data_1.groupby(["Время отправки гео"]).cumcount() + 1
data_2["timestamp_rownum"] = data_2.groupby(["Время отправки гео"]).cumcount() + 1
data_3["timestamp_rownum"] = data_3.groupby(["Время отправки гео"]).cumcount() + 1

In [50]:
data_1['Акселерометр'] = data_1['Акселерометр'].str.replace("\n", "").apply(lambda x: eval(x) if x is not None else [np.nan, np.nan, np.nan])
data_2['Акселерометр'] = data_2['Акселерометр'].str.replace("\n", "").apply(lambda x: eval(x) if x is not None else [np.nan, np.nan, np.nan])
data_3['Акселерометр'] = data_3['Акселерометр'].str.replace("\n", "").apply(lambda x: eval(x) if x is not None else [np.nan, np.nan, np.nan])

In [51]:
data_1[["acc_x", "acc_y", "acc_z"]] = data_1['Акселерометр'].to_list()
data_2[["acc_x", "acc_y", "acc_z"]] = data_2['Акселерометр'].to_list()
data_3[["acc_x", "acc_y", "acc_z"]] = data_3['Акселерометр'].to_list()

In [53]:
merged_data_1 = data_1.merge(data_2, on=["Время отправки гео", "timestamp_rownum"], how='inner')

In [54]:
merged_data_2 = data_1.merge(data_3, on=["Время отправки гео", "timestamp_rownum"], how='inner')